In [2]:
import cv2
import numpy as np
import requests
from pyzbar.pyzbar import decode
from gtts import gTTS
import os
import re
from playsound import playsound
import easyocr
import time
import easyocr
import pyttsx3

In [3]:
def speak(text, lang='es'):
    
    # Create a gTTS object
    tts = gTTS(text=text, lang=lang)

    # Save the audio file
    tts.save("output.mp3")
    time.sleep(0.2)
    playsound("output.mp3")
    time.sleep(0.5)
    os.remove("output.mp3")


def extract_nutrient(nutrition_facts, nutrient):
    nutrient_match = re.search(rf"{nutrient} ([\d.]+) g", nutrition_facts)
    if nutrient_match:
        nutrient_value = float(nutrient_match.group(1))
        return nutrient_value
    else:
        return None

def speak_text(text):
    tts = gTTS(text=text, lang='en')
    tts.save("ocr_.mp3")
    playsound("ocr_.mp3")
    os.remove("ocr_.mp3")

def read_title_with_easyocr(frame):
    es_reader = easyocr.Reader(['es'])
    result = es_reader.readtext(frame)
    for detection in result:
        if len(detection)>1 and detection[2]>0.95:
            return (detection[1].lower())
            #return speak(detection[1])
    
def output_speech():
    cp = cv2.VideoCapture(0)
    cp.set(3, 640)
    cp.set(4, 640)

    processed_codes = set()  
    start_time = time.time()

    while True: 
        success, img = cp.read()
        if not success:
            continue
        
        cv2.imshow('Camera', img)

        barcode_detected = False
        for barcode in decode(img):
            barcode_detected = True
            myData = barcode.data.decode('utf-8')
          
            if myData not in processed_codes:  
                processed_codes.add(myData)  
                
                api_key = "3nb94dtsnold4n9glj3fib7iix7h6a" 
                api_url = f"https://api.barcodelookup.com/v3/products?barcode={myData}&formatted=y&key={api_key}"
                response = requests.get(api_url)

                if response.status_code == 200:
                    json_data = response.json()
                    #print(json_data)

                    title = json_data.get('products')[0].get('title')
                    conversation_text = f"You got {title}. ¿Do you want more information about this food?"
                    

                    speak(conversation_text)
                else:
                    print("Failed to fetch data from API")

                start_time = time.time() 

        elapsed_time = time.time() - start_time
        if not barcode_detected and elapsed_time > 5:
            title_text = read_title_with_easyocr(img)
            if title_text:
                #print(title_text)
                speak(f"You got {title_text}. ¿Do you want more information about this food?")
            start_time = time.time()  

        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break

    cp.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    output_speech()